In [3]:
import unicodedata
import re
import time
from deep_translator import GoogleTranslator
import pandas as pd
from requests.exceptions import SSLError
import json

In [23]:
def remove_vietnamese_accent(text, special_char_map):
    """
    Loại bỏ dấu tiếng Việt và xử lý các ký tự đặc biệt

    Args:
        text (str): Văn bản cần xử lý
        special_char_map (dict, optional): Bảng chuyển đổi ký tự đặc biệt.
            Mặc định xử lý chữ 'đ'/'Đ'
    """

    # Loại bỏ dấu
    text = unicodedata.normalize("NFD", text)
    text = "".join(char for char in text if unicodedata.category(char) != "Mn")

    # Thay thế các ký tự đặc biệt
    for char, replacement in special_char_map.items():
        text = text.replace(char, replacement)

    return text


def remove_unnecessary_words(province, outlier_province_map, outlier_provinces):
    # Lowercase để chuẩn hóa
    province = province.lower()

    # Bước 1: Loại bỏ các từ không cần thiết
    province = re.sub(r"\b(thanh pho|pho|province|city)\b", "", province)

    # Bước 2: Xử lý các trường hợp đặc biệt Hà Tĩnh (ha tinh))
    for special_case in outlier_provinces:
        if special_case in province:
            province = special_case  # Case Ha Tinh
        else:
            province = re.sub(r"\btinh\b", "", province)

    # Bước 3: Xử lý các trường hợp Đắk Lắk (dac lak -> dak lak)
    for old_name, new_name in outlier_province_map.items():
        if old_name in province:
            province = new_name

    # Bước 4: Loại bỏ các ký tự đặc biệt và khoảng trắng
    province = re.sub(
        r"[-–]", " ", province
    )  # Case Thua Thien - Hue and Ba Ria - Vung Tau
    province = " ".join(province.split())

    return province.title()


def clean_province(province):
    # Bước 1: Loại bỏ dấu tiếng Việt
    outlier_char_map = {"đ": "d", "Đ": "D", "ð": "d", "Ð": "D"}

    province = remove_vietnamese_accent(province, outlier_char_map)

    # Bước 2: Loại bỏ các từ không cần thiết
    outlier_province_map = {
        "dac lak": "dak lak",
        "lau dai dac lac": "dak lak",
        "tan an": "long an",
        "hin tin": "binh dinh",
        "phong thu hang hai": "hai phong"
    }
    outlier_provinces = ["ha tinh"]
    # print(province)

    province = remove_unnecessary_words(
        province, outlier_province_map, outlier_provinces
    )

    return province


def contains_special_chars(text, include_vietnamese=False):
    if include_vietnamese:
        vietnamese_chars = r"[àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ]"
        cleaned_text = re.sub(vietnamese_chars, "", text)
        return bool(re.search(r"[^a-zA-Z\s]", cleaned_text))
    return bool(re.search(r"[^a-zA-Z\s]", text))


# def translate_text(text, target_lang="vi"):
#     translator = GoogleTranslator(target=target_lang)
#     # Tự động phát hiện và dịch
#     translated = translator.translate(text)
#     time.sleep(0.5)

#     return {"original": text, "translated": translated}


def translate_text(text, target_lang="vi", max_retries=10):
    translator = GoogleTranslator(target=target_lang)
    retries = 0

    while retries < max_retries:
        try:
            # Tự động phát hiện và dịch
            translated = translator.translate(text)
            time.sleep(0.5)
            return {"original": text, "translated": translated}

        except SSLError as e:
            retries += 1
            print(f"[Retry {retries}/{max_retries}] SSL Error: {e}")
            time.sleep(2)  # nghỉ 2s rồi thử lại

    raise RuntimeError(f"Failed to translate after {max_retries} retries due to SSL errors.")


def process_province(text):
    if contains_special_chars(text, include_vietnamese=False):
        # Translate text with special characters
        result = translate_text(text)
        if isinstance(result, dict):
            print(result["original"] + " " + result["translated"])
            translated = result["translated"]
            # Check if translated text still contains special characters
            return (
                "Others"
                if contains_special_chars(translated, include_vietnamese=True)
                else translated
            )
    return text


In [24]:
df = pd.read_csv(r"C:\Users\vulx8\Downloads\Tất cả đơn hàng-2025-08-14-13_37.csv")

df["Clean Province"] = (
    df["Province"]
    .apply(clean_province)
    .apply(process_province)
    .apply(clean_province)
)

with open("province_mapping.json", "r", encoding="utf-8") as f:
    province_mapping = json.load(f)

df["Clean Province"] = df["Clean Province"].map(province_mapping)

[Retry 1/10] SSL Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=vi&sl=auto&q=%E3%83%92%E3%83%B3%E3%83%86%E3%82%A3%E3%83%B3%E7%9C%81 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)')))
ヒンティン省 Tỉnh Hin Tin
[Retry 1/10] SSL Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=vi&sl=auto&q=%E6%B5%B7%E9%98%B2 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)')))
[Retry 2/10] SSL Error: HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: /m?tl=vi&sl=auto&q=%E6%B5%B7%E9%98%B2 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl

In [7]:
import pandas as pd

df = pd.read_csv(r"/home/chithanh/Downloads/2026-02-20T16-20_export.csv")
df.to_excel(r"/home/chithanh/Downloads/2026-02-20T16-20_export.xlsx", index=False)

In [6]:
translator = GoogleTranslator(target="vi")
text = "กรุงฮานอย"
translated = translator.translate(text)

translated

'Hà Nội'